## Датасет собран из базы данных переписи 1994 года и содержит данные о доходах.
### Информация о данных:
* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, * Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
* salary: >50K,<=50K

## Проведите анализ данных при помощи Pandas выполнив поставленные задачи.
#### 

In [2]:
import pandas as pd

In [3]:
# загружаем датасет
df = pd.read_csv("./data/adult.data.csv")
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**1. Посчитайте, сколько мужчин и женщин (признак *sex*) представлено в этом датасете**

In [193]:
sex_count = df.loc[df['sex'].isin(['Male', 'Female']), 'sex'].count()
print(sex_count)

32561


**2. Каков средний возраст мужчин (признак *age*) по всему датасету?**

In [6]:
avg_age_man = df.loc[df['sex']=='Male','age'].mean()
print(avg_age_man)

39.43354749885268


**3. Какова доля граждан Соединенных Штатов (признак *native-country*)?**

In [226]:
american_eagle = df['native-country'].value_counts(normalize=True).reset_index(name='proportion')
print(american_eagle[american_eagle['native-country']=='United-States'])

  native-country  proportion
0  United-States    0.895857


**4-5. Рассчитайте среднее значение и среднеквадратичное отклонение возраста тех, кто получает более 50K в год (признак *salary*) и тех, кто получает менее 50K в год**

In [276]:
print(df.loc[df['salary']=='<=50K','age'].agg(['mean', 'std']).reset_index(name='<=50K'))
print(df.loc[df['salary']=='>50K','age'].agg(['mean', 'std']).reset_index(name='>50K'))

mean    38.581647
std     13.640433
Name: age, dtype: float64

**6. Правда ли, что люди, которые получают больше 50k, имеют минимум высшее образование? (признак *education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters* или *Doctorate*)**

In [16]:
edu = df.loc[df['salary']=='>50K']['education'].isin([ 'Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate'])
df.groupby(edu)['education'].count()

education
False    3306
True     4535
Name: education, dtype: int64

**7. Выведите статистику возраста для каждой расы (признак *race*) и каждого пола. Используйте *groupby* и *describe*. Найдите таким образом максимальный возраст мужчин расы *Asian-Pac-Islander*.**

In [65]:
df.groupby(['race','sex']).describe()
max_age_asian = (df[(df['race']=='Asian-Pac-Islander') & (df['sex']=='Male')])['age'].max()
print(max_age_asian)

90


**8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак *marital-status*)? Женатыми считаем тех, у кого *marital-status* начинается с *Married* (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.**

In [98]:
family_status = df.loc[(df['salary']=='>50K') &(df['sex']=='Male'), 'marital-status'].isin(['Married-civ-spouse', 'Married-spouse-absent', 'Married-AF-spouse'])

res = family_status.replace({True: 'Женат', False: 'Холост'}).value_counts(normalize=True)*100
res.reset_index(name='% male >50K')


,marital-status,% male >50K
0,Женат,89.537676
1,Холост,10.462324


**9. Какое максимальное число часов человек работает в неделю (признак *hours-per-week*)? Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?**

In [17]:
max_per_week = df['hours-per-week'].max()
count_people = df[df['hours-per-week']==max_per_week]
rich = count_people[count_people['salary']=='>50K']
perc = (rich.shape[0] / count_people.shape[0]) * 100
result = {
    'Question':['max hours per week', 'count_people', 'count who more 50K', 'percentage'],
    'Answers':[max_per_week, count_people.shape[0], rich.shape[0], perc]
}
pd.DataFrame(result)


,Question,Answers
0,max hours per week,99.000000
1,count_people,85.000000
2,count who more 50K,25.000000
3,percentage,29.411765


**10. Посчитайте среднее время работы (*hours-per-week*) зарабатывающих мало и много (*salary*) для каждой страны (*native-country*).**

In [277]:
df.groupby(['native-country', 'salary'])['hours-per-week'].mean()

native-country  salary
?               <=50K     40.164760
                >50K      45.547945
Cambodia        <=50K     41.416667
                >50K      40.000000
Canada          <=50K     37.914634
                            ...    
United-States   >50K      45.505369
Vietnam         <=50K     37.193548
                >50K      39.200000
Yugoslavia      <=50K     41.600000
                >50K      49.500000
Name: hours-per-week, Length: 82, dtype: float64

**11.Сгруппируйте людей по возрастным группам *young*, *adult*, *retiree*, где:**
* *young* соответствует 16-35 лет
* *adult* - 35-70 лет
* *retiree* - 70-100 лет

**Проставьте название соответсвтуещей группы для каждого человека в новой колонке AgeGroup**

In [19]:
import numpy as np

cond = {
        'young': (df['age'] >= 16) & (df['age'] < 35),
        'adult': (df['age'] >= 35) & (df['age'] < 70),
        'retiree': (df['age'] >= 70) & (df['age'] < 100)
       }
df['AgeGroup'] = np.select(cond.values(), cond.keys(), default='unknown')
print(df)

       age         workclass  fnlwgt   education  education-num  \
0       39         State-gov   77516   Bachelors             13   
1       50  Self-emp-not-inc   83311   Bachelors             13   
2       38           Private  215646     HS-grad              9   
3       53           Private  234721        11th              7   
4       28           Private  338409   Bachelors             13   
...    ...               ...     ...         ...            ...   
32556   27           Private  257302  Assoc-acdm             12   
32557   40           Private  154374     HS-grad              9   
32558   58           Private  151910     HS-grad              9   
32559   22           Private  201490     HS-grad              9   
32560   52      Self-emp-inc  287927     HS-grad              9   

           marital-status         occupation   relationship   race     sex  \
0           Never-married       Adm-clerical  Not-in-family  White    Male   
1      Married-civ-spouse    Exec-manag

**12-13. Определите количество зарабатывающих >50K в каждой из возрастных групп (колонка AgeGroup), а также выведите название возрастной группы, в которой чаще зарабатывают больше 50К (>50K)**

In [21]:
salary_50 = df.loc[df['salary']=='>50K', 'AgeGroup'].value_counts()
print(salary_50)
rich_age_group = df.loc[df['salary']=='>50K', 'AgeGroup']
rich_age_group.value_counts().idxmax()



AgeGroup
adult      6240
young      1488
retiree     113
Name: count, dtype: int64


'adult'

**14. Сгруппируйте людей по типу занятости (колонка occupation) и определите количество людей в каждой группе. После чего напишите функциюю фильтрации filter_func, которая будет возвращать только те группы, в которых средний возраст (колонка age) не больше 40 и в которых все работники отрабатывают более 5 часов в неделю (колонка hours-per-week)**

In [23]:
group_by_occ = df.groupby(df['occupation'])['age'].count().reset_index(name='count')
def filter_func(df):
    avg_cond = df['age'].mean()<40
    hours_per_week = (df['hours-per-week']>5).all()
    return (avg_cond & hours_per_week)
grupa = df.groupby(['occupation']).filter(filter_func)
grupa['occupation'].unique()

array(['Armed-Forces'], dtype=object)